In [1]:
import tensorflow as tf
import hls4ml
import plotting
import matplotlib.pyplot as plt
import yaml
%matplotlib inline

In [2]:
hls4ml.__version__

'0.5.1'

In [3]:
model = tf.keras.models.load_model('mnist_model2.h5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 28, 28, 8)         80        
_________________________________________________________________
max1 (MaxPooling2D)          (None, 14, 14, 8)         0         
_________________________________________________________________
batch1 (BatchNormalization)  (None, 14, 14, 8)         32        
_________________________________________________________________
relu1 (Activation)           (None, 14, 14, 8)         0         
_________________________________________________________________
drop1 (Dropout)              (None, 14, 14, 8)         0         
_________________________________________________________________
conv2 (Conv2D)               (None, 14, 14, 8)         584       
_________________________________________________________________
max2 (MaxPooling2D)          (None, 7, 7, 8)          

In [4]:
config = hls4ml.utils.config_from_keras_model(model, granularity='name')
config['Model']['Precision'] = 'ap_fixed<16,6>'
config['Model']['Strategy'] = 'Resource'
config['Model']['ReuseFactor'] = 250
config['LayerName']['soft']['Strategy'] = 'Stable'
plotting.print_dict(config)
hls_model = hls4ml.converters.convert_from_keras_model(model,
                                                       output_dir='model_2/hls4ml_prj',
                                                       fpga_part='xcu250-figd2104-2L-e',
                                                       io_type = 'io_stream',
                                                       hls_config = config)

Interpreting Sequential
Topology:
Layer name: conv1_input, layer type: Input
Layer name: conv1, layer type: Conv2D
  -> Activation (relu), layer name: conv1
Layer name: max1, layer type: MaxPooling2D
Layer name: batch1, layer type: BatchNormalization
Layer name: relu1, layer type: Activation
Layer name: conv2, layer type: Conv2D
  -> Activation (relu), layer name: conv2
Layer name: max2, layer type: MaxPooling2D
Layer name: batch2, layer type: BatchNormalization
Layer name: relu2, layer type: Activation
Layer name: conv3, layer type: Conv2D
  -> Activation (linear), layer name: conv3
Layer name: max3, layer type: MaxPooling2D
Layer name: batch3, layer type: BatchNormalization
Layer name: relu3, layer type: Activation
Layer name: dense2, layer type: Dense
  -> Activation (relu), layer name: dense2
Layer name: batch5, layer type: BatchNormalization
Layer name: relu5, layer type: Activation
Layer name: dense5, layer type: Dense
  -> Activation (linear), layer name: dense5
Layer name: soft

In [5]:
hls_model.compile()

Writing HLS project
Done


In [6]:
import pandas as pd
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")
Y_train = train["label"]
X_train = train.drop(labels = ["label"],axis = 1) 

del train 
X_train = X_train / 255.0
test = test / 255.0
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)
Y_train = to_categorical(Y_train, num_classes = 10)
random_seed = 2
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=random_seed)

In [7]:
import numpy as np
def accuracy(y_pred, y_true):
    cnt = 0
    for i in range(len(y_pred)):
        if y_pred[i] == y_true[i]:
            cnt += 1
    return cnt/len(y_pred)

In [8]:
keras_pred = model.predict(X_val)
hls_pred = hls_model.predict(X_val)
keras_pred = np.argmax(keras_pred, axis = 1)
y_pred = np.argmax(hls_pred, axis = 1)
y_true = np.argmax(Y_val, axis = 1)

print('hls accuracy:', accuracy(y_pred, y_true))
print('keras acuracy:', accuracy(keras_pred, y_true))

hls accuracy: 0.930952380952381
keras acuracy: 0.9357142857142857


In [9]:
hls_model.build(csim=True,synth=True,export=False)

{'CSimResults': [[0.0126953,
   1.0,
   0.0224609,
   0.0205078,
   0.0410156,
   0.0166016,
   0.0195313,
   0.0634766,
   0.0253906,
   0.0224609]],
 'EstimatedClockPeriod': '4.366',
 'BestLatency': '7485',
 'WorstLatency': '7485',
 'IntervalMin': '1801',
 'IntervalMax': '7201',
 'BRAM_18K': '1042',
 'DSP48E': '1977',
 'FF': '47819',
 'LUT': '81045',
 'URAM': '0',
 'AvailableBRAM_18K': '5376',
 'AvailableDSP48E': '12288',
 'AvailableFF': '3456000',
 'AvailableLUT': '1728000',
 'AvailableURAM': '1280'}

In [10]:
hls4ml.report.read_vivado_report('model_1/hls4ml_prj/')

Found 1 solution(s) in model_1/hls4ml_prj//myproject_prj.
Reports for solution "solution1":

C SIMULATION RESULT:
INFO: [SIM 2] *************** CSIM start ***************
INFO: [SIM 4] CSIM will launch GCC as the compiler.
   Compiling ../../../../myproject_test.cpp in debug mode
   Compiling ../../../../firmware/myproject.cpp in debug mode
   Generating csim.exe
INFO: Unable to open input/predictions file, using default input.
0.0126953 1 0.0224609 0.0205078 0.0410156 0.0166016 0.0195313 0.0634766 0.0253906 0.0224609 
INFO: Saved inference results to file: tb_data/csim_results.log
INFO: [SIM 1] CSim done with 0 errors.
INFO: [SIM 3] *************** CSIM finish ***************

SYNTHESIS REPORT:
== Vivado HLS Report for 'myproject'
* Date:           Sun Mar  7 16:46:26 2021

* Version:        2019.2 (Build 2704478 on Wed Nov 06 22:10:23 MST 2019)
* Project:        myproject_prj
* Solution:       solution1
* Product family: virtexuplus
* Target device:  xcu250-figd2104-2L-e


== Perform

In [12]:
hls4ml.report.read_vivado_report('model_2/hls4ml_prj/')

Found 1 solution(s) in model_2/hls4ml_prj//myproject_prj.
Reports for solution "solution1":

C SIMULATION RESULT:
INFO: [SIM 2] *************** CSIM start ***************
INFO: [SIM 4] CSIM will launch GCC as the compiler.
   Compiling ../../../../myproject_test.cpp in debug mode
   Compiling ../../../../firmware/myproject.cpp in debug mode
   Generating csim.exe
INFO: Unable to open input/predictions file, using default input.
0.0126953 1 0.0224609 0.0205078 0.0410156 0.0166016 0.0195313 0.0634766 0.0253906 0.0224609 
INFO: Saved inference results to file: tb_data/csim_results.log
INFO: [SIM 1] CSim done with 0 errors.
INFO: [SIM 3] *************** CSIM finish ***************

SYNTHESIS REPORT:
== Vivado HLS Report for 'myproject'
* Date:           Thu Mar 11 21:13:51 2021

* Version:        2019.2 (Build 2704478 on Wed Nov 06 22:10:23 MST 2019)
* Project:        myproject_prj
* Solution:       solution1
* Product family: virtexuplus
* Target device:  xcu250-figd2104-2L-e


== Perform